In [14]:
%reload_ext autoreload
%autoreload 2

In [15]:
import torch
import torchvision

In [17]:
from k12libs.utils.nb_easy import k12ai_model_summary
# net = torchvision.models.resnet18()
net = torchvision.models.densenet121()
k12ai_model_summary(net, input_size=(32, 3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 112, 112]           9,408
       BatchNorm2d-2         [32, 64, 112, 112]             128
              ReLU-3         [32, 64, 112, 112]               0
         MaxPool2d-4           [32, 64, 56, 56]               0
       BatchNorm2d-5           [32, 64, 56, 56]             128
              ReLU-6           [32, 64, 56, 56]               0
            Conv2d-7          [32, 128, 56, 56]           8,192
       BatchNorm2d-8          [32, 128, 56, 56]             256
              ReLU-9          [32, 128, 56, 56]               0
           Conv2d-10           [32, 32, 56, 56]          36,864
      BatchNorm2d-11           [32, 96, 56, 56]             192
             ReLU-12           [32, 96, 56, 56]               0
           Conv2d-13          [32, 128, 56, 56]          12,288
      BatchNorm2d-14          [32, 128,

In [19]:
from k12libs.utils.nb_easy import k12ai_start_html, W3URL
k12ai_start_html(f'{W3URL}/drawnet.html?jfile=complex', height=780, flask=True)

 * Serving Flask app "k12libs.utils.nb_easy" (lazy loading)


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [13]:
回家from torchsummary import summary


1


AttributeError: 'list' object has no attribute 'size'

In [18]:
a = '''
plain_net { name : \"hproject_model_name\" layer {
					 conv {
					  name : \"Conv2d_sevppsr6kn\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : CONV2D
					  inputs: 'x'
					  outputs : \"Flatten_ebarro3rr\"
					  layer_params { 
					 	 in_channels : \"3\"  out_channels : \"64\"  kernel_size : \"13\"  stride : \"2\"  padding : \"1\"  dilation : \"1\"  groups : \"1\"  bias : \"True\" 
					 } 
					 }
					 }layer {
					 linear {
					  name : \"Linear_c039phayy0\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : LINEAR
					  inputs: \"Flatten_ebarro3rr\" 
					  outputs : \"Output_raw8c877zn\"
					  layer_params { 
					 	 in_features : \"5184\"  out_features : \"10\"  bias : \"True\" 
					 } 
					 }
					 }layer {
					 vulkan {
					  name : \"Flatten_ebarro3rr\"
					  layer_builder : \"NNTorchLayer\"
					  layer_mode : FLATTEN
					  inputs: \"Conv2d_sevppsr6kn\" 
					  outputs : \"Linear_c039phayy0\"
					  layer_params { 
					 	 start_dim : \"1\"  end_dim : \"-1\" 
					 } 
					 }
					 } } 
'''
print(a)


plain_net { name : "hproject_model_name" layer {
					 conv {
					  name : "Conv2d_sevppsr6kn"
					  layer_builder : "NNTorchLayer"
					  layer_mode : CONV2D
					  inputs: 'x'
					  outputs : "Flatten_ebarro3rr"
					  layer_params { 
					 	 in_channels : "3"  out_channels : "64"  kernel_size : "13"  stride : "2"  padding : "1"  dilation : "1"  groups : "1"  bias : "True" 
					 } 
					 }
					 }layer {
					 linear {
					  name : "Linear_c039phayy0"
					  layer_builder : "NNTorchLayer"
					  layer_mode : LINEAR
					  inputs: "Flatten_ebarro3rr" 
					  outputs : "Output_raw8c877zn"
					  layer_params { 
					 	 in_features : "5184"  out_features : "10"  bias : "True" 
					 } 
					 }
					 }layer {
					 vulkan {
					  name : "Flatten_ebarro3rr"
					  layer_builder : "NNTorchLayer"
					  layer_mode : FLATTEN
					  inputs: "Conv2d_sevppsr6kn" 
					  outputs : "Linear_c039phayy0"
					  layer_params { 
					 	 start_dim : "1"  end_dim : "-1" 
					 } 
					 }
					 } 